In [2]:
# Input files after preparation

receptor_aptamer = "./inputs/4dii_clean.pdbqt"     # Complex (receptor with aptamer)
azides = './inputs/compounds.npy'

In [3]:
# Name of the flexible residue with it's index  

flexible_residues = 'DT3'

## Modules

In [4]:
# Import packages

from collections.abc import Iterable

import os
import re
import copy

import numpy as np

# https://github.com/rdkit/rdkit
import rdkit
from rdkit import Chem, Geometry
from rdkit.Chem import AllChem, Draw, rdFMCS
from rdkit.Chem.Draw import IPythonConsole 
from IPython.display import display, Image
#from rdkit.Chem import rdBase, rdMolAlign, rdDistGeom, rdMolDescriptors
from rdkit.Chem.rdDistGeom import ETKDG
p = ETKDG()
p.verbose = True

# https://github.com/oddt/oddt
import oddt

## Functions

In [5]:
class Nucl:
    
    def __init__(self, name=None, atom=None, residue=None, chain=None):
        self.name = name
        self.atom = atom
        self.residue = residue
        self.chain = chain        


class NuclParser(Nucl):
    
    def __file__(self, filename):         
        self.filename = filename
        
    def __list__(self):
        self.index = index
        self.all_lines = all_lines
    
    def ___dic__(self, ind_name):
        self.ind_name = ind_name
    
    def lineIndexer(self, filename):    
        self.filename = filename
        
        with open(self.filename) as f:
            lines = f.readlines()
            for idx, line in enumerate(lines):
                (idx, line.strip('\r\n'))
                
        index = []   # Set indexes to lines from file
        for i in range(idx):
            index.append(i)
        
        all_lines = []    # Make list of strings from file
        for l in lines:
            all_lines.append(l.strip('\r\n'))
            
        self.index = index
        self.all_lines = all_lines

In [6]:
class FileReader:
    
    def __init__(self, name=None):
        import os
        name = os.path.splitext('receptor_filename')[0]
        self.name = name
    
    def __list__(self, index, all_lines):
        self.index = index
        self.all_lines = all_lines

    
    def readFile(self, filename=None):
        self.filename = filename
        f = open(self.filename)
        lines = f.read()
        all_lines = lines.strip('\n')
        if len(all_lines)==1:
            self.filename[0].split('\r')
            warnings.warn('Only 1 line read from file, splitting on "\r"') 
        file_list = []
        file_list.append(all_lines) 
        f.close()  
        return(file_list)   

In [7]:
def parsePDBQT(mol, filename, **kwards):
       
#    Read topology from a PDBQT file
#    
#    Creates the following Attributes:
#     - atom ids (serial)
#     - atom types
#     - atom names
#     - altLocs
#     - resnames
#     - chainIDs (becomes segid)
#     - resids
#     - record_types (ATOM/HETATM)
#     - icodes
#     - coords (all coordinates)
#     - x
#     - y
#     - z
#     - occupancies
#     - tempfactors
#     - charges
#     - atomtypes
#     - resiues (resname + chaunIDs + resids)
#
#
#
#    1 -  6         Record name   "CRYST1"
#    7 - 15         Real(9.3)     a              a (Angstroms).
#    16 - 24        Real(9.3)     b              b (Angstroms).
#    25 - 33        Real(9.3)     c              c (Angstroms).
#    34 - 40        Real(7.2)     alpha          alpha (degrees).
#    41 - 47        Real(7.2)     beta           beta (degrees).
#    48 - 54        Real(7.2)     gamma          gamma (degrees).
#
#    1 -  6         Record name   "ATOM  "
#    7 - 11         Integer       serial       Atom  serial number.
#    13 - 16        Atom          name         Atom name.
#    17             Character     altLoc       Alternate location indicator. IGNORED
#    18 - 21        Residue name  resName      Residue name.
#    22             Character     chainID      Chain identifier.
#    23 - 26        Integer       resSeq       Residue sequence number.
#    27             AChar         iCode        Code for insertion of residues. IGNORED
#    31 - 38        Real(8.3)     x            Orthogonal coordinates for X in Angstroms.
#    39 - 46        Real(8.3)     y            Orthogonal coordinates for Y in Angstroms.
#    47 - 54        Real(8.3)     z            Orthogonal coordinates for Z in Angstroms.
#    55 - 60        Real(6.2)     occupancy    Occupancy.
#    61 - 66        Real(6.2)     tempFactor   Temperature  factor.
#    67 - 76        Real(10.4)    partialChrg  Gasteiger PEOE partial charge *q*.
#    79 - 80        LString(2)    atomType     AutoDOCK atom type *t*.



    record_types = []
    serials = []
    names = []
    altlocs = []
    resnames = []
    chainids = []
    resids = []
    icodes = []
    coords = []
    x = []
    y = []
    z = []
    occupancies = []
    tempfactors = []
    charges = []
    atomtypes = []
    residues = []
 
    with open(filename) as f:
        for line in f:
            line = line.strip()
            if not line.startswith(('ATOM', 'HETATM')):
                continue
                
            record_types.append(line[:6].strip())
            serials.append(int(line[6:11]))
            names.append(line[12:16].strip())
            altlocs.append(line[16:17].strip())
            resnames.append(line[17:21].strip())
            chainids.append(line[21:22].strip())
            resids.append(int(line[22:26]))
            icodes.append(line[26:27].strip())
            coords.append(str(line[30:54].strip()))
            x.append(str(line[30:38].strip()))
            y.append(str(line[38:46].strip()))
            z.append(str(line[46:54].strip()))
            occupancies.append(str(line[54:60]))
            tempfactors.append(str(line[60:66]))
            charges.append(str(line[66:76]))
            atomtypes.append(line[77:80].strip())
            residues.append(line[12:26].strip())
            
    n_atoms = len(serials)
    
    mol.record_types = record_types
    mol.serials = serials
    mol.names = names
    mol.altlocs = altlocs
    mol.resnames = resnames
    mol.chainids = chainids
    mol.resids = resids
    mol.icodes = icodes
    mol.coords = coords
    mol.x = x
    mol.y = y
    mol.z = z
    mol.occupancies = occupancies
    mol.tempfactors = tempfactors
    mol.charges = charges
    mol.atomtypes = atomtypes
    mol.n_atoms = n_atoms  
    mol.residues = residues
    
    return mol


In [8]:
def MolFromPDBQTBlock(block, sanitize=True, removeHs=False):
    
#    Read PDBQT block to a RDKit Molecule
#
#    Parameters:
#    
#        block: string
#            Residue name which explicitly pint to a ligand(s).
#        sanitize: bool (default=True)
#            Should the sanitization be performed
#        removeHs: bool (default=True)
#            Should hydrogens be removed when reading molecule.
#
#    Returns:
#    
#        mol: rdkit.Chem.rdchem.Mol
#            Molecule read from PDBQT


    pdb_lines = []
    name = ''
    data = {}
    for line in block.split('\n'):
        # Get all know data from REMARK section
        if line[:12] == 'REMARK  Name':
            name = line[15:].strip()
        elif line[:18] == 'REMARK VINA RESULT':
            tmp = line[19:].split()
            data['vina_affinity'] = tmp[0]
            data['vina_rmsd_lb'] = tmp[1]
            data['vina_rmsd_ub'] = tmp[2]

        # no more data to collect
        if line[:4] != 'ATOM':
            continue

        pdb_line = line[:56]
        pdb_line += '1.00  0.00           '

        # Do proper atom type lookup
        atom_type = line[71:].split()[1]
        if atom_type == 'A':
            atom_type = 'C'
        elif atom_type[:1] == 'O':
            atom_type = 'O'
        elif atom_type[:1] == 'H':
            atom_type = 'H'
            if removeHs:
                continue
        elif atom_type == 'NA':
            atom_type = 'N'

        pdb_lines.append(pdb_line + atom_type)
        
    mol = Chem.MolFromPDBBlock('\n'.join(pdb_lines), sanitize=False)
    if sanitize:
        Chem.SanitizeMol(mol)
    else:
        Chem.GetSSSR(mol)
        
    # Reorder atoms using serial
    new_order = sorted(range(mol.GetNumAtoms()),
                       key=lambda i: (mol.GetAtomWithIdx(i)
                                      .GetPDBResidueInfo()
                                      .GetSerialNumber()))
    mol = Chem.RenumberAtoms(mol, new_order)

    # Properties must be set on final copy of Mol, RenumberAtoms purges data
    mol.SetProp('_Name', name)
    for k, v in data.items():
        mol.SetProp(str(k), str(v))

    return mol

In [9]:
# Modified molecules To PDBQT with charges

def ModPDBQTAtomLines(mod_mol, donors=True, acceptors=True):
    
#    Create a list with PDBQT atom lines for each atom in molecule. 
#    Donors and acceptors are given as a list of atom indices.   
    
    if donors:
        patt = Chem.MolFromSmarts('[$([N&!H0&v3,N&!H0&+1&v4,n&H1&+0,$([$([Nv3](-C)(-C)-C)]),'
                          '$([$(n[n;H1]),'
                          '$(nc[n;H1])])]),'
                          # Guanidine can be tautormeic - e.g. Arginine
                          '$([NX3,NX2]([!O,!S])!@C(!@[NX3,NX2]([!O,!S]))!@[NX3,NX2]([!O,!S])),'
                          '$([O,S;H1;+0])]')
        donors = list(map(lambda x: x[0],
                          mod_mol.GetSubstructMatches(patt, maxMatches=mod_mol.GetNumAtoms())))
        
        
    if acceptors:    
        patt = Chem.MolFromSmarts('[$([O;H1;v2]),'
                              '$([O;H0;v2;!$(O=N-*),'
                              '$([O;-;!$(*-N=O)]),'
                              '$([o;+0])]),'
                              '$([n;+0;!X3;!$([n;H1](cc)cc),'
                              '$([$([N;H0]#[C&v4])]),'
                              '$([N&v3;H0;$(Nc)])]),'
                              '$([F;$(F-[#6]);!$(FC[F,Cl,Br,I])])]')
        acceptors = list(map(lambda x: x[0],
                             mod_mol.GetSubstructMatches(patt, maxMatches=mod_mol.GetNumAtoms())))

    full_mod_pdbqt_lines = []

    mod_atom_lines = [mod_line.replace('HETATM', 'ATOM  ')
                  for mod_line in Chem.MolToPDBBlock(mod_mol).split('\n')
                  if mod_line.startswith('HETATM') or mod_line.startswith('ATOM')]

    mod_pdbqt_lines = []
    atom_types_mod_pdbqt_lines = []

    charges = []
    for m_idx, m_atom in enumerate(mod_mol.GetAtoms()):
        mod_pdbqt_line = mod_atom_lines[m_idx][:70]
        mod_pdbqt_lines.append(mod_pdbqt_line)


        # Get atom type
        mod_pdbqt_line += ' '
        atomicnum = m_atom.GetAtomicNum()
        if atomicnum == 6 and m_atom.GetIsAromatic():
            mod_pdbqt_line += 'A'
        elif atomicnum == 7 and m_idx in acceptors:
            mod_pdbqt_line += 'NA'
        elif atomicnum == 8 and m_idx in acceptors:
            mod_pdbqt_line += 'OA'
        elif atomicnum == 1 and m_atom.GetNeighbors()[0].GetIdx() in donors:
            mod_pdbqt_line += 'HD'
        else:
            mod_pdbqt_line += m_atom.GetSymbol()
        atom_types_mod_pdbqt_lines.append(mod_pdbqt_line)


        # Gasteiger Charges 
        charge = 0.
        fields = ['_MMFF94Charge', '_GasteigerCharge', '_TriposPartialCharge']
        for f in fields:
            if m_atom.HasProp(f):
                charge = m_atom.GetDoubleProp(f)
                break


        AllChem.ComputeGasteigerCharges(mod_mol)
        atoms_a = mod_mol.GetAtoms()
        for idx_atom in atoms_a:
            idx_atom_a = idx_atom.GetIdx()
            gasteiger = float(mod_mol.GetAtomWithIdx(idx_atom_a).GetProp('_GasteigerCharge'))
            charges.append(str("%.3f" % gasteiger).rjust(6))

    gasteiger_mod_pdbqt_lines = []        

    for (mod_pdbqt_line, g_charge) in zip(mod_pdbqt_lines, charges):
        mod_pdbqt_line += g_charge
        gasteiger_mod_pdbqt_lines.append(mod_pdbqt_line)

    for pdb_mod_pdbqt_line, gasteiger_mod_pdbqt_line in zip(atom_types_mod_pdbqt_lines, gasteiger_mod_pdbqt_lines):
        pdbqt_atom_types = pdb_mod_pdbqt_line[70:]
        full_mod_pdbqt_lines.append(pdb_mod_pdbqt_line[:70] + gasteiger_mod_pdbqt_line[70:] + pdbqt_atom_types)


    return full_mod_pdbqt_lines

In [10]:
def check_mol(mol):
    checked_mol = Chem.MolToSmiles(mol).replace('-','')
    return Chem.MolFromSmiles(checked_mol)

def neutralizeRadicals(mol):
    for a in mol.GetAtoms():
        if a.GetNumRadicalElectrons() == 1 or a.GetFormalCharge()==1:
            a.SetNumRadicalElectrons(0)         
            a.SetFormalCharge(0)
            

In [11]:
def MolToPDBQTBlock(mol, mcs_init=False, flexible=True, ngly=False, addHs=False, computeCharges=True, VinaScores=False, FlexResidue=True):
    
#    Write RDKit Molecule to a PDBQT block
#
#    Parameters:
#    
#        mol: rdkit.Chem.rdchem.Mol
#            Molecule with a protein ligand complex
#        raw: PDBQT lines with flexible atoms of the residue.
#        flexible: bool (default=True)
#            Should the molecule encode torsions. Ligands should be flexible,
#            proteins in turn can be rigid.
#        addHs: bool (default=False)
#            The PDBQT format requires at least polar Hs on donors. By default Hs
#            are added.
#        computeCharges: bool (default=False)
#            Should the partial charges be automatically computed. If the Hs are
#            added the charges must and will be recomputed. If there are no
#            partial charge information, they are set to 0.0.
#        VinaScores: bool (default=False)
#            Write pdbqt of AutoDockVina results
#        FlexResidue: bool (default=True)
#            Write pdbqt of flexible N-glycosidic bond residue
#
#    Returns:
#    
#        block: str
#            String wit PDBQT encoded molecule


    # Make a copy of molecule
    
    mol = Chem.Mol(mol) 
    
    # If flexible molecule contains multiple fragments write them separately
    if flexible and len(Chem.GetMolFrags(mol)) > 1:
        jointed = ''.join(MolToPDBQTBlock(frag, flexible=flexible, addHs=addHs,
                                       computeCharges=computeCharges)
                       for frag in Chem.GetMolFrags(mol, asMols=True))
        return jointed
    
    # Identify donors and acceptors for atom typing
    # Acceptors
    patt1 = Chem.MolFromSmarts('[$([O;H1;v2]),'
                              '$([O;H0;v2;!$(O=N-*),'
                              '$([O;-;!$(*-N=O)]),'
                              '$([o;+0])]),'
                              '$([n;+0;!X3;!$([n;H1](cc)cc),'
                              '$([$([N;H0]#[C&v4])]),'
                              '$([N&v3;H0;$(Nc)])]),'
                              '$([F;$(F-[#6]);!$(FC[F,Cl,Br,I])])]')

    # Donors
    patt2 = Chem.MolFromSmarts('[$([N&!H0&v3,N&!H0&+1&v4,n&H1&+0,$([$([Nv3](-C)(-C)-C)]),'
                              '$([$(n[n;H1]),'
                              '$(nc[n;H1])])]),'
                              # Guanidine can be tautormeic - e.g. Arginine
                              '$([NX3,NX2]([!O,!S])!@C(!@[NX3,NX2]([!O,!S]))!@[NX3,NX2]([!O,!S])),'
                              '$([O,S;H1;+0])]')

    
    acceptors = list(map(lambda x: x[0],
                         mol.GetSubstructMatches(patt1, maxMatches=mol.GetNumAtoms())))
        
    donors = list(map(lambda x: x[0],
                      mol.GetSubstructMatches(patt2, maxMatches=mol.GetNumAtoms())))
    
    if addHs:
        mol = Chem.AddHs(mol, addCoords=True, onlyOnAtoms=donors, )
    if addHs or computeCharges:
        AllChem.ComputeGasteigerCharges(mol)
    
    atom_lines = ModPDBQTAtomLines(mol)
    assert len(atom_lines) == mol.GetNumAtoms()
    
    new_pdbqt_lines = []
    
    if FlexResidue:
        new_pdbqt_lines.append(("BEGIN_RES "+mol.resnames[0])+" "+mol.chainids[0]+(str(mol.resids[0]).rjust(3)))
        
        
    # Vina scores
    if VinaScores:
        if (mol.HasProp('vina_affinity') and mol.HasProp('vina_rmsd_lb') and
                mol.HasProp('vina_rmsd_lb')):
            new_pdbqt_lines.append('REMARK VINA RESULT:  ' +
                               ('%.1f' % float(mol.GetProp('vina_affinity'))).rjust(8) +
                               ('%.3f' % float(mol.GetProp('vina_rmsd_lb'))).rjust(11) +
                               ('%.3f' % float(mol.GetProp('vina_rmsd_ub'))).rjust(11))
        
            new_pdbqt_lines.append('REMARK  Name = ' +
                           (mol.GetProp('_Name') if mol.HasProp('_Name') else ''))
    
 
    # Find rotatable bonds
    if flexible:
        if mcs_init:           
            rot_bond = Chem.MolFromSmarts('[!$(*#*)&!D1&!$(C(F)(F)F)&'
                                          '!$(C(Cl)(Cl)Cl)&'
                                          '!$(C(Br)(Br)Br)&'
                                          '!$(C([CH3])([CH3])[CH3])&'
                                          '!$([CD3](=[N,O,S])-!@[#7,O,S!D1])&'
                                          '!$([#7,O,S!D1]-!@[CD3]=[N,O,S])&'
                                          '!$([CD3](=[N+])-!@[#7!D1])&'
                                          '!$([#7!D1]-!@[CD3]=[N+])]-!@[!$(*#*)&'
                                          '!D1&!$(C(F)(F)F)&'
                                          '!$(C(Cl)(Cl)Cl)&'
                                          '!$(C(Br)(Br)Br)&'
                                          '!$(C([CH3])([CH3])[CH3])]')            

            ngly_bond = Chem.MolFromSmarts('[C]-!@[n&X3&x2]')    # N-glycosidic bond
                       
            core_substruct = Chem.MolFromSmarts(mcs_init.smartsString)
            substruct_atoms = list(mol.GetSubstructMatches(core_substruct))
            ngly_atoms = list(mol.GetSubstructMatches(ngly_bond))
            
            bond_atoms = list(mol.GetSubstructMatches(rot_bond))
            
            get_ngly = []
            
            for i, j in ngly_atoms:
                for s in substruct_atoms:
                    if i in s and j in s:
                        get_ngly.append((i, j))
                        
            bond_atoms.extend(get_ngly)            
                        
            num_torsions = len(bond_atoms)
            
            # Active torsions header
            new_pdbqt_lines.append('REMARK  %i active torsions:' % num_torsions)
            new_pdbqt_lines.append('REMARK  status: (\'A\' for Active; \'I\' for Inactive)')
            for i, (a1, a2) in enumerate(bond_atoms):
                new_pdbqt_lines.append('REMARK%5.0i  A    between atoms: _%i  and  _%i'
                                   % (i + 1, a1 + 1, a2 + 1))
                
                
            # Fragment molecule on bonds to rigid fragments
            bond_ids = [mol.GetBondBetweenAtoms(a1, a2).GetIdx()
                        for a1, a2 in bond_atoms]
            if bond_ids:
                mol_rigid_frags = Chem.FragmentOnBonds(mol, bond_ids, addDummies=False)
            else:
                mol_rigid_frags = mol
            frags = list(Chem.GetMolFrags(mol_rigid_frags))
            
            # Start writting the lines with ROOT
            new_pdbqt_lines.append('ROOT')
            frag = frags.pop()
            for idx in frag:
                new_pdbqt_lines.append(atom_lines[idx])
            new_pdbqt_lines.append('ENDROOT')
 
            def weigh_frags(frag):
                # Sort by the fragment size and the number of bonds (secondary)
                num_bonds = 0
                # bond_weight = 0
                for a1, a2 in bond_atoms:
                   # print(a1, a2)
                    if a1 in frag or a2 in frag:
                        num_bonds += 1
                return len(frag), num_bonds  # Bond_weight
            
            frags = sorted(frags, key=weigh_frags)            
                   
        else:    
            
            rot_bond = Chem.MolFromSmarts('[!$(*#*)&!D1&!$(C(F)(F)F)&'
                                          '!$(C(Cl)(Cl)Cl)&'
                                          '!$(C(Br)(Br)Br)&'
                                          '!$(C([CH3])([CH3])[CH3])&'
                                          '!$([CD3](=[N,O,S])-!@[#7,O,S!D1])&'
                                          '!$([#7,O,S!D1]-!@[CD3]=[N,O,S])&'
                                          '!$([CD3](=[N+])-!@[#7!D1])&'
                                          '!$([#7!D1]-!@[CD3]=[N+])]-!@[!$(*#*)&'
                                          '!D1&!$(C(F)(F)F)&'
                                          '!$(C(Cl)(Cl)Cl)&'
                                          '!$(C(Br)(Br)Br)&'
                                          '!$(C([CH3])([CH3])[CH3])]')
            
            if ngly:
                rot_bond = Chem.MolFromSmarts('[C]-!@[n,N&X3&x2]')       
                
            bond_atoms = list(mol.GetSubstructMatches(rot_bond))
            
            num_torsions = len(bond_atoms)
        
            # Active torsions header
            new_pdbqt_lines.append('REMARK  %i active torsions:' % num_torsions)
            new_pdbqt_lines.append('REMARK  status: (\'A\' for Active; \'I\' for Inactive)')
            for i, (a1, a2) in enumerate(bond_atoms):
                new_pdbqt_lines.append('REMARK%5.0i  A    between atoms: _%i  and  _%i'
                                   % (i + 1, a1 + 1, a2 + 1))
                
            # Fragment molecule on bonds to rigid fragments
            bond_ids = [mol.GetBondBetweenAtoms(a1, a2).GetIdx()
                        for a1, a2 in bond_atoms]
            if bond_ids:
                mol_rigid_frags = Chem.FragmentOnBonds(mol, bond_ids, addDummies=False)
            else:
                mol_rigid_frags = mol
            frags = list(Chem.GetMolFrags(mol_rigid_frags))
            
            def weigh_frags(frag):
                # Sort by the fragment size and the number of bonds (secondary)
                num_bonds = 0
                # bond_weight = 0
                for a1, a2 in bond_atoms:
                   # print(a1, a2)
                    if a1 in frag or a2 in frag:
                        num_bonds += 1
                return len(frag), num_bonds  # Bond_weight               
            
            frags = sorted(frags, key=weigh_frags)
            
            # Start writting the lines with ROOT
            new_pdbqt_lines.append('ROOT')
            frag = frags.pop(0)
            for idx in frag:
                new_pdbqt_lines.append(atom_lines[idx])
            new_pdbqt_lines.append('ENDROOT')            
            
        # Build the tree of torsions usign DFS algorithm. Keep track of last
        # route with following variables to move down the tree and close branches
        branch_queue = []
        current_root = frag
        old_roots = [frag]  
        visited_frags = []
        visited_bonds = []
        while len(frags) > len(visited_frags):
            end_branch = True
            for frag_num, frag in enumerate(frags):
                for bond_num, (a1, a2) in enumerate(bond_atoms):
                    if (frag_num not in visited_frags and
                        bond_num not in visited_bonds and
                        (a1 in current_root and a2 in frag or
                         a2 in current_root and a1 in frag)):
                        # direction of bonds is important
                        if a1 in current_root:
                            bond_dir = '%2.0i  %2.0i' % (a1 + 1, a2 + 1)
                        else:
                            bond_dir = '%2.0i  %2.0i' % (a2 + 1, a1 + 1)
                        new_pdbqt_lines.append(('BRANCH  %s' % bond_dir).rjust(3))
                        for idx in frag:
                            new_pdbqt_lines.append(atom_lines[idx])
                        branch_queue.append(('ENDBRANCH  %s' % bond_dir).rjust(3))
        
        
                        # Overwrite current root and stash previous one in queue
                        old_roots.append(current_root)
                        current_root = frag
        
                        # Remove used elements from stack
                        visited_frags.append(frag_num)
                        visited_bonds.append(bond_num)
        
                        # Mark that we dont want to end branch yet
                        end_branch = False
                        break
                    else: 
                        continue
                    break   # Break the outer loop as well
        
            if end_branch:
                new_pdbqt_lines.append(branch_queue.pop())
                if old_roots:
                    current_root = old_roots.pop()
            # Close opened branches if any is open
        while len(branch_queue):
            new_pdbqt_lines.append(branch_queue.pop())
        if VinaScores:
            new_pdbqt_lines.append('TORSDOF %i' % num_torsions)
        if FlexResidue:
            new_pdbqt_lines.append(("END_RES "+mol.resnames[0])+" "+mol.chainids[0]+(str(mol.resids[0]).rjust(3)))
                                                                                    
    else:
        new_pdbqt_lines.extend(atom_lines)
    
    return '\n'.join(new_pdbqt_lines)

In [12]:
def chemdraw(smiles, include_atom_numbers = False):
    mol = Chem.MolFromSmiles(smiles)
    AllChem.Compute2DCoords(mol)
    display(Draw.MolToImage(mol, includeAtomNumbers = include_atom_numbers))
    
def mol_with_atom_index( mol ):
    atoms = mol.GetNumAtoms()
    for idx in range( atoms ):
        mol.GetAtomWithIdx( idx ).SetProp( 'molAtomMapNumber', str( mol.GetAtomWithIdx( idx ).GetIdx() ) )
    return mol

def Make3D(m3d, conf_id = -1):
    rdkit.Chem.rdmolops.AddHs(m3d)
    AllChem.Compute2DCoords(m3d)
    Chem.AllChem.EmbedMolecule(m3d)
    AllChem.MMFFOptimizeMolecule(m3d, maxIters=500, nonBondedThresh=200, confId = conf_id)
    return m3d

## Preparing raw molecules

In [13]:
# Prepare rigid residue


fmt = {'ATOM': ("ATOM  {serial:5d} {name:<4.4s} {resName:<4.4s}"     # PDBQT format
                 "{chainID:1.1s}{resSeq:4d} "
                 "   {x:8.3f}{y:8.3f}{z:8.3f}{occupancy:6.2f}"
                 "{tempFactor:6.2f}    {charge:< 1.3f} {element:<2.2s}\n")}


sel = NuclParser()
sel.lineIndexer(receptor_aptamer)
comp = rdkit.Chem.rdchem.Mol
parsePDBQT(comp, receptor_aptamer)

a = sel.all_lines
copy_a = copy.copy(a)
letters = re.findall(r'\d*\D+', flexible_residues)    # Name of the flexible residue (both DNA and RNA)
digits = re.findall('[0-9]{1,3}', flexible_residues)      # Number of the flexible residue

cut_ind = []        # Find the flexible residue in all residues of the complex
for i, j in enumerate(comp.residues, 2):
    if (j.split()[1] == str(letters).strip("'[]'") and j.split()[3] == str(digits).strip("'[]'")):
        cut_ind.append(i)

cut_lines = []
for s in cut_ind:
    cut_lines.append(copy_a[s])

flex = []    
for ng in cut_lines:
    match = re.search(r"C1['\*]", ng)     # Find N-glycosidic bond in the flexible residue
    if match:
        # print (ng)
        ngly = cut_lines.index(ng)
        
flex.append(cut_lines[ngly:])
flex_r = [item for sublist in flex for item in sublist]

flex_raw = []
for i, flr in enumerate(flex_r, 1):
    ri = flr.split()[1]
    ri = i
    flex_raw.append(fmt['ATOM'].format(serial=int(ri), name=str(flr.split()[2]), resName=str(flr.split()[3]), chainID=str(flr.split()[4]), resSeq=int(flr.split()[5]), x=float(flr.split()[6]), y=float(flr.split()[7]), z=float(flr.split()[8]), occupancy=float(flr.split()[9]), tempFactor=float(flr.split()[10]), charge=float(flr.split()[11]), element=str(flr.split()[12])))

with open('raw_' + flexible_residues + '_flex.pdbqt', 'w') as output_cut:    # Write part of the flexible residue as rigid
    for cut in flex_raw:
        output_cut.write("%s" % cut)
                
rigid = [l for l in a if l.startswith('ATOM')]
rigid_lines = [x for x in rigid if x not in flex_r]

with open('rigid_complex.pdbqt', 'w') as output_rigid:
    for rigid in rigid_lines:
      #  print(rigid)
        output_rigid.write(rigid + '\n')

#read('raw_flex.pdbqt')

In [14]:
mol_to_pdbqt = Chem.Mol()
block = "\n".join(flex_raw)
mol_from_pdbqt = MolFromPDBQTBlock(block)

#mol_from_pdbqt

In [15]:
# Identify donors and acceptors for atom typing

# Acceptors
patt1 = Chem.MolFromSmarts('[$([O;H1;v2]),'
                          '$([O;H0;v2;!$(O=N-*),'
                          '$([O;-;!$(*-N=O)]),'
                          '$([o;+0])]),'
                          '$([n;+0;!X3;!$([n;H1](cc)cc),'
                          '$([$([N;H0]#[C&v4])]),'
                          '$([N&v3;H0;$(Nc)])]),'
                          '$([F;$(F-[#6]);!$(FC[F,Cl,Br,I])])]')

# Donors
patt2 = Chem.MolFromSmarts('[$([N&!H0&v3,N&!H0&+1&v4,n&H1&+0,$([$([Nv3](-C)(-C)-C)]),'
                          '$([$(n[n;H1]),'
                          '$(nc[n;H1])])]),'
                          # Guanidine can be tautormeic - e.g. Arginine
                          '$([NX3,NX2]([!O,!S])!@C(!@[NX3,NX2]([!O,!S]))!@[NX3,NX2]([!O,!S])),'
                          '$([O,S;H1;+0])]')

In [16]:
# Make flexible N-glycosidic bond

raw = FileReader()
for line in raw.readFile('raw_' + flexible_residues + '_flex.pdbqt'):
            
    acceptors = list(map(lambda x: x[0],
                 mol_from_pdbqt.GetSubstructMatches(patt1, maxMatches=mol_from_pdbqt.GetNumAtoms())))
    donors = list(map(lambda x: x[0],
              mol_from_pdbqt.GetSubstructMatches(patt2, maxMatches=mol_from_pdbqt.GetNumAtoms())))
    
    mol_to_pdbqt = MolToPDBQTBlock(mol_from_pdbqt, ngly=True)#, 'raw_' + flexible_residues + '_flex.pdbqt')
    
    with open(flexible_residues + '_flex.pdbqt', 'w') as write_object:
        write_object.write(mol_to_pdbqt)

#read("raw_flex_out.pdb")

## Alignment

In [17]:
az = np.load(azides, allow_pickle=True)

In [18]:
core_mod = Chem.MolFromPDBFile(flexible_residues + '_flex.pdbqt')
smiles = [Chem.MolToSmiles(a) for a in az[:5]]

In [19]:
# Create modified nucleotides

ms = []

for s in smiles:
    mol = Chem.MolFromSmiles(s)
    query = Chem.MolFromSmiles('N=[N+]=[N-]')
    replacement = Chem.MolFromSmiles('[N]2C=C(c1cn(C)c(=O)[nH]c1=O)N=N2')

    mod_mol = Chem.ReplaceSubstructs(mol, query, replacement, 0) 
    Chem.SanitizeMol(mod_mol[0])
    neutralizeRadicals(mod_mol[0])
    ms.append(mod_mol[0])

In [20]:
# Generate 3d structures (in case you don't have them already)

mhs = [Chem.AddHs(x) for x in ms]
mols = []

for x in mhs:
    try: 
        AllChem.EmbedMolecule(x, AllChem.ETKDG())
        mols.append(Chem.RemoveHs(x))
    except Exception:
        pass

In [21]:
#len(mols)

In [22]:
mol_count = 0

probs = []
rmsVs = []

# Find the MCS (maximum common substructure)

for probeMol in mols:
    mcs = rdFMCS.FindMCS([core_mod, probeMol], threshold = 0.9, completeRingsOnly=True, timeout=5000, verbose=True, bondCompare=rdkit.Chem.rdFMCS.BondCompare.CompareAny, ringMatchesRingOnly=True)
    print(mcs.smartsString)

    # Align everything to the first molecule
    patt = Chem.MolFromSmarts(mcs.smartsString)
    refMol = core_mod
    refMatch = refMol.GetSubstructMatch(patt)    
    mv = probeMol.GetSubstructMatch(patt)
    try:
        rms = AllChem.AlignMol(probeMol, refMol, atomMap=list(zip(mv, refMatch)), maxIters=500000)
        rmsVs.append(rms)
        probs.append(probeMol)
        print(mol_count)
        print(' ')
        mol_count += 1
        
    except (ValueError, RuntimeError) as error:
        continue

results = list(zip(probs, rmsVs))

[#7&R]1(-&!@[#6&!R])-,:;@[#6&R](-,=;!@[#8&!R])-,:;@[#7&R]-,:;@[#6&R](-,:;@[#6&R]-,:;@[#6&R]-,:;@1)-,=;!@[#8&!R]
0
 
[#7&R]1(-&!@[#6&!R])-,:;@[#6&R](-,=;!@[#8&!R])-,:;@[#7&R]-,:;@[#6&R](-,:;@[#6&R]-,:;@[#6&R]-,:;@1)-,=;!@[#8&!R]
1
 
[#7&R]1(-&!@[#6&!R])-,:;@[#6&R](-,=;!@[#8&!R])-,:;@[#7&R]-,:;@[#6&R](-,:;@[#6&R]-,:;@[#6&R]-,:;@1)-,=;!@[#8&!R]
2
 
[#7&R]1(-&!@[#6&!R])-,:;@[#6&R](-,=;!@[#8&!R])-,:;@[#7&R]-,:;@[#6&R](-,:;@[#6&R]-,:;@[#6&R]-,:;@1)-,=;!@[#8&!R]
3
 
[#7&R]1(-&!@[#6&!R])-,:;@[#6&R](-,=;!@[#8&!R])-,:;@[#7&R]-,:;@[#6&R](-,:;@[#6&R]-,:;@[#6&R]-,:;@1)-,=;!@[#8&!R]
4
 


In [23]:
count = 0

for i in enumerate(results):
    with open('./mod_nucls/' + 'flexible_residues' + '_flex_mod_v5_' + str(i[0]) + '.pdbqt', 'w') as wf:
        try:
            wf.write(MolToPDBQTBlock(i[1][0], mcs_init=mcs))
            print('./mod_nucls/' + 'flexible_residues' + '_flex_mod_v5_' + str(i[0]) + '.pdbqt' + ' done')
            count += 1
        except Exception:
            pass
            

./mod_nucls/flexible_residues_flex_mod_v5_0.pdbqt done
./mod_nucls/flexible_residues_flex_mod_v5_1.pdbqt done
./mod_nucls/flexible_residues_flex_mod_v5_2.pdbqt done
./mod_nucls/flexible_residues_flex_mod_v5_3.pdbqt done
./mod_nucls/flexible_residues_flex_mod_v5_4.pdbqt done
